# Introduction

## Project Overview
In this project, we delve into the task of cleaning a dataset from Airbnb. Airbnb, a prominent player in the online accommodation booking space, offers a wealth of data that can provide insightful information into rental trends, pricing patterns, host activities, and guest preferences.

## Objective
The primary objective of this data cleaning exercise is to prepare the dataset for accurate and efficient analysis. Data cleaning is a critical step in the data analysis process as it involves rectifying or removing incorrect, incomplete, or irrelevant data. By ensuring the quality and integrity of our data, we can guarantee more reliable and valid analysis results.

## Dataset Description
The dataset encompasses various features related to Airbnb listings, such as location details, pricing, host information, reviews, and availability. However, like many real-world datasets, it contains its share of challenges, including missing values, outliers, and inconsistencies that need to be addressed.

## Approach
Our approach to cleaning this dataset involves several steps:
- Identifying and handling missing values.
- Correcting data types and formats.
- Detecting and treating outliers.
- Ensuring data consistency and accuracy.

By the end of this project, we aim to have a clean, well-structured dataset that is ready for further analysis and can be used to derive meaningful insights about the Airbnb market.

## Source
Airbnb Open Data on Kaggle:
- https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata/data

---

In [2]:
import pandas as pd

file_path = '/Users/frostyalpaca/Desktop/airbnb_data_cleaning/data/Airbnb_Open_Data.csv'

airbnb_df = pd.read_csv(file_path)

airbnb_df.head()

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/3888476061.py:5: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pd.read_csv(file_path)


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [3]:
airbnb_df.isnull().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified               289
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    214
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15893
reviews per month                  15879
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

License and house_rules have a huge amount of missing data. After that is last review and reviews per month. These features may not be that useful and will have to be addressed. 
Looking license and house_rules columns, there is not much we can do and should be dropped since.

In [4]:
# Columns to be dropped
columns_to_drop = ['license','house_rules']

# Dropping the columns
airbnb_data_cleaned = airbnb_df.drop(columns=columns_to_drop)

For the remaining columns, it's time to check what is the percentage of the missing values to the whole column to see if we can possibily impute the missing values if they are small enough.

In [11]:
# Creating a function that gives the percentage of missing values in each column

def calculate_missing_percentage(airbnb_data_cleaned):

    # Calculate the number of missing values per column
    missing_counts = airbnb_data_cleaned.isnull().sum()

    # Calculate the total number of rows
    total_rows = len(airbnb_data_cleaned)

    # Calculate the percentage of missing values
    missing_percentages = (missing_counts / total_rows) * 100

    return missing_percentages

# Applying the function to the Airbnb dataset
missing_percentages = calculate_missing_percentage(airbnb_data_cleaned)

# Display
missing_percentages

id                                 0.000000
NAME                               0.243686
host id                            0.000000
host_identity_verified             0.281701
host name                          0.000000
neighbourhood group                0.000000
neighbourhood                      0.000000
lat                                0.000000
long                               0.000000
country                            0.000000
country code                       0.127692
instant_bookable                   0.102348
cancellation_policy                0.074081
room type                          0.000000
Construction year                  0.207621
price                              0.240762
service fee                        0.266105
minimum nights                     0.398670
number of reviews                  0.178378
last review                       15.490638
reviews per month                 15.476991
review rate number                 0.317767
calculated host listings count  

The idea is that any percentage of missing values under 10% will be used for imputation possibily. We are going to by column by column the meet the criteria of a missing value percentage under 10%

For the 'NAME' column we will have a placeholder for the 'NaN' value. The placeholder will be 'Unknown'.

In [13]:
# Imputing missing values in 'NAME' with a placeholder 'Unknown'
airbnb_data_cleaned['NAME'].fillna('Unknown', inplace=True)

For the 'host_identity_verified' column we will use the mode to impute the values of 'NaN'. The reason why it is a categorical column, either 'verified' or 'unconfirmed'. The mode will revealed what is the highest frequency being used and likely guess the 'NaN' values.

In [14]:
# Find the mode for 'host_identity_verified'
host_mode_value = airbnb_data_cleaned['host_identity_verified'].mode()[0]

# Impute missing values with the most
airbnb_data_cleaned['host_identity_verified'].fillna(host_mode_value, inplace=True)

Now we are moving to the next column, 'host name'. Host name is a personal identifier, we can't just make up a name for that person. Instead we will use a placeholder of 'Unknown' to fill the missing values.

In [15]:
# Imputing the missing values in 'host name' with a placeholder of 'Unknown'
airbnb_data_cleaned['host name'].fillna('Unknown', inplace=True)

Now looking at 'neighbourhood group', I think it is best to use 'Unknown' again. Now there is an arguement that we can use the 'neighbourhood' column to guess 'neighbourhood group', but due to time constraints of this project, we will go forward with the 'Unknown' method.

In [16]:
# Imputing missing values in 'neighbourhood group' with a placeholder 'Unknown'
airbnb_data_cleaned['neighbourhood group'].fillna('Unknown', inplace=True)

Same as with the 'neighbourhood group', I am opting for the 'Unknown' approach for replacing the missing values

In [17]:
# Imputing the missing value in 'neighbourhood' with 'Unknown'
airbnb_data_cleaned['neighbourhood'].fillna('Unknown', inplace=True)

For lat and long column there is multiple options we can choose. Using means, median, or mode alone is off the table because the geographical locations can be very far from the latitude and longitude. But we can use the neighbourhood or country to help impute latitude and longitude based on the mean or median of that area. Another one is try to use outside sources like Google Maps to find it. But due to time constraint we will either use removing the rows or just leaving them missing. I'm going to choose removing the missing value rows since it would only be 8 rows for longitutde and latitude. This will also ensure that geographic analysis would be more accurate.

In [18]:
# Dropping rows where either 'lat' or 'long' column has missing values
airbnb_data_cleaned.dropna(subset=['lat', 'long'], inplace=True)

Due to time constraints of the project, we will replace 'NaN' with 'Unknown' for the country column.

In [19]:
# Imputing the missing value in 'neighbourhood' with 'Unknown'
airbnb_data_cleaned['country'].fillna('Unknown', inplace=True)

For 'country code' we can impute based on 'country' column. We can also use a placeholder, remove rows, or leave it missing. Since I do not believe 'country code' is a significant column, I will choose the placeholder.

In [22]:
# Imputing the missing value in 'neighbourhood' with 'Unknown'
airbnb_data_cleaned['country code'].fillna('Unknown', inplace=True)

For 'instant_bookable', I will choose the mode, whichever has a higher frequency (True or False), I will impute for the missing values.

In [27]:
# Finding the most common value in 'instant_bookable' column
mode_instant_bookable = airbnb_data_cleaned['instant_bookable'].mode()[0]

# Imputing the missing values with the mode value
airbnb_data_cleaned['instant_bookable'].fillna(mode_instant_bookable, inplace=True)

The two options I see with 'cancellation_policy' is either to use a placeholder or use the mode. Using a placeholder will create a fourth category. Using the mode may cause bias within the data. I will choose mode for the sake of practice.

In [31]:
# Finding the most common value in 'cancellation_policy columns
mode_cancellation_policy = airbnb_data_cleaned['cancellation_policy'].mode()[0]

# Imputing the missing values with the mode value
airbnb_data_cleaned['cancellation_policy'].fillna(mode_cancellation_policy, inplace=True)

In [32]:
airbnb_data_cleaned.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified                0
host name                             0
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
country code                          0
instant_bookable                      0
cancellation_policy                   0
room type                             0
Construction year                   213
price                               247
service fee                         273
minimum nights                      409
number of reviews                   183
last review                       15892
reviews per month                 15878
review rate number                  326
calculated host listings count      319
availability 365                    448
dtype: int64

In [34]:
unique = airbnb_data_cleaned['Construction year'].unique()

In [35]:
print(unique)

[2020. 2007. 2005. 2009. 2013. 2015. 2004. 2008. 2010. 2019. 2018. 2006.
 2016. 2017. 2021. 2003. 2011. 2012. 2022. 2014.   nan]


In [36]:
airbnb_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102591 entries, 0 to 102598
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102591 non-null  int64  
 1   NAME                            102591 non-null  object 
 2   host id                         102591 non-null  int64  
 3   host_identity_verified          102591 non-null  object 
 4   host name                       102591 non-null  object 
 5   neighbourhood group             102591 non-null  object 
 6   neighbourhood                   102591 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102591 non-null  object 
 10  country code                    102591 non-null  object 
 11  instant_bookable                102591 non-null  bool   
 12  cancellation_pol

Since Construction year is a float64 Dtype, using 'Unknown' as a placeholder is off the table because it will interfere calculations. Think using the mode can cause a bit of bias, since there is a lot of years construction could've been. I can't leave the missing values because the 'NaN' value can also interfere with the calculation. Therefore I am the decision to remove the rows with missing values.

In [37]:
# Remove rows where 'Construction year' is missing
airbnb_data_cleaned = airbnb_data_cleaned.dropna(subset=['Construction year'])

Price is Dtype is object. Before we start cleaning this column, we want to convert it to a float first so it can be used in calculations

In [41]:
# Remove the dollar sign and convert to numeric
airbnb_data_cleaned['price'] = airbnb_data_cleaned['price'].replace('[\$,]', '', regex=True).astype(float)

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/2110275434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data_cleaned['price'] = airbnb_data_cleaned['price'].replace('[\$,]', '', regex=True).astype(float)


For price, I choose median to replace the missing values, so they are less affected by outliers if I used the mean.

In [45]:
# Calculate the median of the 'price' column
median_price = airbnb_data_cleaned['price'].median()

# Impute missing values with the median
airbnb_data_cleaned['price'].fillna(median_price, inplace=True)

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/1093754133.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data_cleaned['price'].fillna(median_price, inplace=True)


Same as price, we will need to first convert service dtype to float

In [49]:
# Remove the dollar sign and covert to numeric
airbnb_data_cleaned['service fee'] = airbnb_data_cleaned['service fee'].replace('[\$,]', '', regex=True).astype(float)

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/975529155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data_cleaned['service fee'] = airbnb_data_cleaned['service fee'].replace('[\$,]', '', regex=True).astype(float)


We will use median again to impute the missing values

In [51]:
# Calculate the median of the 'price' column
median_service_fee = airbnb_data_cleaned['service fee'].median()

# Impute the missing values with the median
airbnb_data_cleaned['service fee'].fillna(median_service_fee, inplace=True)

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/4035015781.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data_cleaned['service fee'].fillna(median_service_fee, inplace=True)


For minimum nights, I want to check first if there is outliers to see if we can use the mean.

In [55]:
# Statistical summary
stats_summary = airbnb_data_cleaned['minimum nights'].describe()
print(stats_summary)

count    101981.000000
mean          8.135113
std          30.561006
min       -1223.000000
25%           2.000000
50%           3.000000
75%           5.000000
max        5645.000000
Name: minimum nights, dtype: float64


There is a -1223 minimum night, which is impossible. Let's remove any values that are below 0, and also there is minimum nights that go beyond a year. We will also remove those

In [59]:
# Remove rows where 'minimum nights' is negative or extremly high
airbnb_data_cleaned = airbnb_data_cleaned[(airbnb_data_cleaned['minimum nights'] > 0) & (airbnb_data_cleaned['minimum nights'] < 365)]

In [60]:
# Statistical summary
stats_summary = airbnb_data_cleaned['minimum nights'].describe()
print(stats_summary)

count    101880.000000
mean          7.669415
std          14.954139
min           1.000000
25%           2.000000
50%           3.000000
75%           5.000000
max         364.000000
Name: minimum nights, dtype: float64


Median may be a better fit than mean for replace the missing values

In [62]:
# Find the median for the 'minimum nights'
minimum_nights_median = airbnb_data_cleaned['minimum nights'].median()

# Impute the missing values with the median
airbnb_data_cleaned['minimum nights'].fillna(minimum_nights_median, inplace=True)

Check the summary statistics of 'number of reviews'

In [65]:
# Statiscal Summary
reviews_stat_summary = airbnb_data_cleaned['number of reviews'].describe()
print(reviews_stat_summary)

count    101700.000000
mean         27.385762
std          49.302647
min           0.000000
25%           1.000000
50%           7.000000
75%          30.000000
max        1024.000000
Name: number of reviews, dtype: float64


For 'number of reviews' I will choose median over mean for misplacing the values since the outliers are pretty high.

In [85]:
# Find the median for 'number of reviews'
review_numbers_median = airbnb_data_cleaned['number of reviews'].median()

# Replace the missing values with the median
airbnb_data_cleaned['number of reviews'].fillna(review_numbers_median, inplace=True)

In this scenario, let's assume that that last review is not important to this analysis. Since this column is missing 15% of its value, we will decided to drop the column instead of removing the rows that are missing value. It's not worth losing 15,000+ rows for a column that does not make

In [68]:
# Dropping the 'last review' column'
airbnb_data_cleaned = airbnb_data_cleaned.drop(columns=['last review'])

The 'review per month', even though is missing 15% of its values, it does represent the average rate of reviews a listing receieves per month. This I believe is important info for analysis. I don't want to either drop the column nor remove the rows with missing values. Instead I will use the median to replace missing values

In [71]:
# Find the median for 'reviews per month'
reviews_per_month_median = airbnb_data_cleaned['reviews per month'].median()

# Impute the missing values with the median
airbnb_data_cleaned['reviews per month'].fillna(reviews_per_month_median, inplace=True)

For 'review rate number', I don't want to use a placeholder of 0 or a string placeholder. Because It can give the individual host a bad review and also using a string placeholder will get in the way of doing calculations. I'm hesistant using mean or median to impute because it can give a bad host a good review rate or vice verses. This can introduce bias. Therefore, I think it will decide to use remove the rows.

In [73]:
# Removing rows where 'review rate number' is missing
airbnb_data_cleaned = airbnb_data_cleaned.dropna(subset=['review rate number'])

Since you own a fraction of a property on airbnb, we need to convert it a int dtype from a float dtype. Also we will impute the missing values with the median value

In [75]:
# Find the median
listing_count_median = airbnb_data_cleaned['calculated host listings count'].median()

# Impute the missing values with the median
airbnb_data_cleaned['calculated host listings count'].fillna(listing_count_median, inplace=True)

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/1726227001.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data_cleaned['calculated host listings count'].fillna(listing_count_median, inplace=True)


In [77]:
#  Now change the Dtype to int 
airbnb_data_cleaned['calculated host listings count'] = airbnb_data_cleaned['calculated host listings count'].astype(int)

/var/folders/bj/l1bnwrwj2gvfmhstjqwrc_7m0000gn/T/ipykernel_4503/1718280252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airbnb_data_cleaned['calculated host listings count'] = airbnb_data_cleaned['calculated host listings count'].astype(int)


For practice, we will remove the missing values from 'availability 365' column 

In [81]:
# Remove the missing value rows
airbnb_data_cleaned = airbnb_data_cleaned.dropna(subset=['availability 365'])

For 'availability 365' we want to change it to a int dtype

In [82]:
# Changing the Dtype to int
airbnb_data_cleaned['availability 365'] = airbnb_data_cleaned['availability 365'].astype(int)

In [83]:
# Now check if we have any missing values

airbnb_data_cleaned.isnull().sum()

id                                0
NAME                              0
host id                           0
host_identity_verified            0
host name                         0
neighbourhood group               0
neighbourhood                     0
lat                               0
long                              0
country                           0
country code                      0
instant_bookable                  0
cancellation_policy               0
room type                         0
Construction year                 0
price                             0
service fee                       0
minimum nights                    0
number of reviews                 0
reviews per month                 0
review rate number                0
calculated host listings count    0
availability 365                  0
dtype: int64

# Conclusion

## Summary of Data Cleaning Process
Throughout this project, we have successfully navigated various data cleaning challenges within the Airbnb dataset. Key steps in our process included handling missing values, correcting data types, addressing outliers, and ensuring overall data integrity. Each of these steps was crucial in transforming the raw dataset into a more analytically valuable form.